In [1]:
import pandas as pd
import numpy as np
import csv
import os
import time

In [2]:
from scipy.stats import multivariate_normal
from gp_pref_elicit_luisa import dataset as data
from gp_pref_elicit_luisa import gaussian_process as GP
from momabs_bayesian import bayes_logistic as bayes_logistic
from gp_pref_elicit_luisa.gp_utilities import utils_user as gp_utils_users
from logistic_user import LogisticDecisionMaker as logistic 

In [3]:
synthetic_pcs = np.genfromtxt('synthetic_pcs_sets/obj4size100.csv', delimiter=',')
output_csv = 'experiments/output-BLR_4-100.csv'
output_regret_csv = 'experiments/regret-csvs/regret-BLR_4-100.csv'
objectives = 4

In [4]:
# initializing dataset class
utils_comparisons = data.DatasetPairwise(num_objectives=objectives)
GP = GP.GPPairwise(num_objectives=objectives)
# making an object of LogisticDecisionMaker class
user_simulation = logistic(no_obj=objectives)

In [5]:
dummy_feature_vector = user_simulation.features(np.zeros(objectives))
len(dummy_feature_vector)

25

In [6]:
ground_utility_dataset = user_simulation.ground_utility(synthetic_pcs)
ground_truth_utility_function_dataset = np.max(ground_utility_dataset)
ground_truth_utility_function_dataset

0.4179002203706368

In [7]:
# generating random points to start with from synthetic pcs 
start_points = synthetic_pcs[np.random.choice(synthetic_pcs.shape[0], size=2, replace=False)]

# comparing the initial points for getting the current best
current_best = user_simulation.exact_compare(start_points[0], start_points[1]) 

ground_truth_utility_current_best = user_simulation.ground_utility(current_best)

synthetic_pcs = [v for v in synthetic_pcs if not np.array_equal(v, start_points[0])]
synthetic_pcs = [v for v in synthetic_pcs if not np.array_equal(v, start_points[1])]

# getting points according to thompson sampling 
# thompson_point = user_simulation.thompson_sampled_point(synthetic_pcs)

# utility_thompson_point = user_simulation.ground_utility(thompson_point)

# # comparing the current best and the thompson sampled point
# current_best = user_simulation.exact_compare(current_best, thompson_point)

# ground_truth_utility_current_best = user_simulation.ground_utility(current_best)
# synthetic_pcs = [v for v in synthetic_pcs if not np.array_equal(v, current_best)]

# exclude_point = user_simulation.exclude_points([current_best])

regret = np.subtract(ground_truth_utility_function_dataset, ground_truth_utility_current_best)


print('Start points: ', start_points, '\n',
      'Current Best Point: ', current_best, '\n',
      'Utility of the Current Best Point: ', ground_truth_utility_current_best, '\n',
      # 'Thompson Sampled Point: ', thompson_point, '\n',
      # 'Utility of the Thompson Point: ', utility_thompson_point, '\n',
      # 'Exclude Point: ', exclude_point, '\n',
      'Regret: ', regret)

Start points:  [[0.91820079 0.30555841 0.22767969 0.04957001 0.09615685]
 [0.30030591 0.30991932 0.66562262 0.29140627 0.53459825]] 
 Current Best Point:  [0.91820079 0.30555841 0.22767969 0.04957001 0.09615685] 
 Utility of the Current Best Point:  [0.27033625] 
 Regret:  [0.14756397]


In [8]:
# loop for BLR with stopping condition: if the current best point is already in the list of excluded points
# terminate the loop else continue
stop = False
counter = 0 # counter variable to track the number of queries
regret_hist = [] # to track and store the regret at each step
last_counter = 0 # to store the total number of queries (which we get from the last iteration)
last_regret = 0 # to store the regret gained from the last iteration

first_run = not os.path.exists(output_csv) # checking if the file exists in the directory

regret_file_check = os.path.exists(output_regret_csv) # checking if the file containing regret values at each step exists

# we output all values for regret gained at each iteration in a new csv file
with open(output_regret_csv, 'a', newline='') as csvfile:
    write_to_regret_csv = csv.writer(csvfile)

    # checking if the file exists already, if not we include the header or else we don't
    if not regret_file_check:
        write_to_regret_csv.writerow(['Step', 'Regret'])

    starting_time = time.time() # starting the timer 

    while not stop:

        next_point_ts= user_simulation.thompson_sampled_point(synthetic_pcs)

        if len(synthetic_pcs) == 0 :
            last_counter = counter
            last_regret = regret
            stop = True
            
     
        current_best = user_simulation.exact_compare(next_point_ts, current_best)
        
        ground_truth_utility_function_next_point_ts = user_simulation.ground_utility(next_point_ts)
        ground_truth_utility_function_current_best = user_simulation.ground_utility(current_best)

        regret = np.subtract(ground_truth_utility_function_dataset, ground_truth_utility_function_current_best)

        # calculating regret for each step
        regret_step = regret
        regret_hist.append(regret) # appending the list to contain regret at every step
        print('Regret at step {}: {}'.format(counter, regret_step))
        write_to_regret_csv.writerow([counter + 1, regret_step]) # writing to csv
        print('Regret history saved')

        exclude_point = user_simulation.exclude_points(synthetic_pcs)
        synthetic_pcs = [v for v in synthetic_pcs if not np.array_equal(v, next_point_ts)]

        # incrementing the counter variable
        counter += 1

        # stop the timer
        ending_time = time.time()
        total_time = ending_time - starting_time
        last_counter = counter
        last_regret = regret
        
        print('Next Point according to thompson sampling: ', next_point_ts, '\n',
              'Current best point: ', current_best, '\n',
              'Ground Truth Utility of Thompson Sampled Point', ground_truth_utility_function_next_point_ts, '\n',
              'Ground Truth Utility of Current best point: ', ground_truth_utility_function_current_best, '\n',
              'Simple Regret: ', regret, '\n',
              'Number of Queries: ', counter, '\n',
              'Computation Time: ', total_time, '\n',
              'Exlcude point: ', exclude_point)
        
        if counter >= 98:
            last_counter = counter 
            last_regret = regret

            # we output the counter and regret of the last iteration to a csv for comparisons
            with open(output_csv, 'a', newline='') as csvfile:
                write_to_csv = csv.writer(csvfile)

                # write the header row only for the first run
                if first_run: # if the file does not exist then we output the header otherwise we output only the data
                    write_to_csv.writerow(['Number of Queries', 'Regret'])
                write_to_csv.writerow([last_counter, last_regret])
            print('Successfully written to csv')

Regret at step 0: [0.14756397]
Regret history saved
Next Point according to thompson sampling:  [0.90320145 0.0198829  0.09082165 0.14265863 0.39399461] 
 Current best point:  [0.91820079 0.30555841 0.22767969 0.04957001 0.09615685] 
 Ground Truth Utility of Thompson Sampled Point [0.13525702] 
 Ground Truth Utility of Current best point:  [0.27033625] 
 Simple Regret:  [0.14756397] 
 Number of Queries:  1 
 Computation Time:  0.00804591178894043 
 Exlcude point:  [0.06009164 0.44159309 0.42786491 0.69390051 0.3698895 ]
Regret at step 1: [0.14756397]
Regret history saved
Next Point according to thompson sampling:  [0.89624628 0.00171051 0.08628315 0.42322941 0.10085516] 
 Current best point:  [0.91820079 0.30555841 0.22767969 0.04957001 0.09615685] 
 Ground Truth Utility of Thompson Sampled Point [0.112819] 
 Ground Truth Utility of Current best point:  [0.27033625] 
 Simple Regret:  [0.14756397] 
 Number of Queries:  2 
 Computation Time:  0.015949010848999023 
 Exlcude point:  [0.060

ValueError: attempt to get argmax of an empty sequence